## Neural Network study of the Bankruptcy Modeling

Kudryavtsev O., Yazici M.

In [1]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

# load data
data = pd.read_csv('data.csv')
data

,X1,X2,X3,X4,X5,inactive_active
0,0.505452,0.037576,0.056000,1.424141,0.750397,0
1,0.523732,0.038025,0.072910,1.379251,0.849812,0
2,0.449485,0.023832,0.021829,1.696345,0.981098,0
3,0.370960,0.046000,0.070648,1.444869,0.984495,0
4,0.391547,-0.158851,0.029688,0.722070,1.179211,0
...,...,...,...,...,...,...
55463,0.351600,-0.004169,0.062183,0.064744,1.149179,1
55464,0.108828,0.014911,0.062648,0.952988,0.179018,1
55465,0.088590,0.025745,0.069360,0.876071,0.182800,1
55466,0.100205,0.021948,0.061845,0.833435,0.177880,1


In [2]:
count_1 = data['inactive_active'].sum()
count_0 = len(data['inactive_active']) - count_1
print('The count and ratio of 1 (active firms) in tha data')
print('count: ',count_1,'rate: ', count_1/(count_1+count_0), 'total data :', count_1+count_0)

The count and ratio of 1 (active firms) in tha data
count:  43411 rate:  0.7826314271291556 total data : 55468


In [3]:
print('The count and ratio of 0 (inactive firms) in tha data')
print('count: ',count_0,'rate: ', count_0/(count_1+count_0), 'total data :', count_1+count_0)

The count and ratio of 0 (inactive firms) in tha data
count:  12057 rate:  0.21736857287084446 total data : 55468


## PART 1: The Statistics

In [4]:
import scipy.stats as stats

# Convert data
X = data.drop(['inactive_active'],axis=1)
y = data['inactive_active']

X

,X1,X2,X3,X4,X5
0,0.505452,0.037576,0.056000,1.424141,0.750397
1,0.523732,0.038025,0.072910,1.379251,0.849812
2,0.449485,0.023832,0.021829,1.696345,0.981098
3,0.370960,0.046000,0.070648,1.444869,0.984495
4,0.391547,-0.158851,0.029688,0.722070,1.179211
...,...,...,...,...,...
55463,0.351600,-0.004169,0.062183,0.064744,1.149179
55464,0.108828,0.014911,0.062648,0.952988,0.179018
55465,0.088590,0.025745,0.069360,0.876071,0.182800
55466,0.100205,0.021948,0.061845,0.833435,0.177880


In [5]:
# The one-way ANOVA tests: 
# The null hypothesis that two or more groups have the same population mean.
fvalue, pvalue = stats.f_oneway(X['X1'], X['X2'], X['X3'], X['X4'], X['X5'])
print(fvalue, pvalue)

if pvalue<0.05:
    print(
    "p-value: {}, The null hypothesis is rejected. There is a difference between at least two variables.".format(
        pvalue))
else:
    print(
    "p-value: {}, The null hypothesis is accepted. There are not any differences among the means of variables.".format(
        pvalue))

60.090312345601234 8.163616061421967e-51
p-value: 8.163616061421967e-51, The null hypothesis is rejected. There is a difference between at least two variables.


In [6]:
# // The Test of Normality //
# The creating a function called normality()
# The null hypothesis that the input data is not from a normal distribution.
def normality(x):
    k2, pvalue = stats.normaltest(x)
    alpha = 1e-3
    
    if pvalue < alpha: # null hypothesis: x comes from a normal distribution
        return('the input data is from a normal distribution')
    else:
        return('the input data is not from a normal distribution') 
    
print(normality(X['X1']))
print(normality(X['X2']))
print(normality(X['X3']))
print(normality(X['X4']))
print(normality(X['X5']))


the input data is from a normal distribution
the input data is from a normal distribution
the input data is from a normal distribution
the input data is from a normal distribution
the input data is from a normal distribution


In [7]:
from scipy.stats import levene

# // Test of Homogeneity of Variances // 
# The leneve's test is used instead of Bartlett’s test
# because our data is from a normal distribution.
# The null hypothesis that all input samples are from populations with equal variances.
stat, p = levene(X['X1'], X['X2'], X['X3'], X['X4'], X['X5'])

print(stat, p)

if pvalue<0.05:
    print(
    "p-value: {}, The null hypothesis is rejected. Not all input samples are from populations with equal variances.".format(
        pvalue))
else:
    print(
    "p-value: {}, The null hypothesis is accepted. All input samples are from populations with equal variances.".format(
        pvalue))

32.0843277943918 8.957646232136972e-27
p-value: 8.163616061421967e-51, The null hypothesis is rejected. Not all input samples are from populations with equal variances.


In [8]:
# Tamhane’s T2 all-pairs comparison test for normally distributed data with unequal variances. 
# Tamhane’s T2 test can be performed for all-pairs comparisons in an one-factorial layout with 
# normally distributed residuals but unequal groups variances. 
# A total of m = k(k-1)/2 hypotheses can be tested. 
# The null hypothesis is tested in the two-tailed test against the alternative hypothesis 

import scikit_posthocs as sp

x = pd.DataFrame({"X1": X['X1'], "X2": X['X2'], "X3": X['X3'], "X4": X['X4'],"X5": X['X5'],})
x = x.melt(var_name='groups', value_name='values')
sp.posthoc_tamhane(x, val_col='values', group_col='groups')

,X1,X2,X3,X4,X5
X1,1.000000e+00,1.837254e-06,0.000000,2.203793e-13,0.000000e+00
X2,1.837254e-06,1.000000e+00,0.231753,0.000000e+00,3.241005e-09
X3,0.000000e+00,2.317527e-01,1.000000,0.000000e+00,0.000000e+00
X4,2.203793e-13,0.000000e+00,0.000000,1.000000e+00,1.115297e-10
X5,0.000000e+00,3.241005e-09,0.000000,1.115297e-10,1.000000e+00


## PART 2: The Neural Net training

In [9]:
import tensorflow as tf
from tensorflow import keras
from tensorflow import compat
from keras.models import Sequential
from keras.layers import Dense
import tensorflow_addons as tfa
import h5

#initial values for the index of splitted data
# n = X.shape[0] = 55465
# The train and test data parts of the data are as follows
# Train: X.loc[a:b] and Test: X.loc[c:d] 
# The lenght of the train data is 4 times of the test data.

n=55465
#a=0; b=(4*n/5)-1; c=4*n/5 ; d=n-1          # Fold 1
#a=n/5; b=n-1; c=0 ; d=n/5-1                # Fold 2
#a=2*n/5; b=n/5-1; c=n/5 ; d=2*n/5-1         # Fold 3              
#a=3*n/5; b=2*n/5-1; c=2*n/5 ; d=3*n/5-1    # Fold 4
a=4*n/5; b=(3*n/5)-1; c=3*n/5 ; d=4*n/5-1  # Fold 5

list1 = list(range(44372,55466))
list2 = list(range(0,33279))
list3 = list(range(33279,44372))

X = X.reindex(list1 + list2 + list3) 
y = y.reindex(list1 + list2 + list3)

X_train, X_test, y_train, y_test = [], [], [], []
X_train, X_test = X.loc[a:b], X.loc[c:d]
y_train, y_test = y.loc[a:b], y.loc[c:d]

count_inactive_test, count_active_test = 0, 0
for j in y_test:
    if j==0:
        count_inactive_test+=1

    if j==1:
        count_active_test+=1
        
print("inactive firms in the test:", count_inactive_test)
print("active firms in the test:", count_active_test)

# Analyze class imbalance in the targets
# 0 and 1 mean inactive, active firms respectively.
counts_1 = y_train.sum()
counts_0 = len(y_train) - counts_1

# The weighting for the imlanabce
weight_for_0 = counts_1 / (counts_0 + counts_1)
weight_for_1 = counts_0 / (counts_0 + counts_1)

# Reshape labels
y_train = tf.one_hot(y_train, depth=2, on_value=1.0, off_value=0.0,axis=-1)
y_test = tf.one_hot(y_test, depth=2, on_value=1.0, off_value=0.0,axis=-1)

# Normalize the data using training set statistics
mean = np.mean(X_train, axis=0)
X_train -= mean
X_test -= mean
std = np.std(X_train, axis=0)
X_train /= std
X_test /= std

# Build a binary classification model
model = Sequential()
model.add(keras.Input(shape=(5,)))
model.add(Dense(3, use_bias=True, activation='relu', kernel_initializer='uniform')) #
model.add(Dense(2, activation="sigmoid", kernel_constraint=keras.constraints.NonNeg())) 


# define the keras model
metrics = [
    keras.metrics.TrueNegatives(name="tn"),
    keras.metrics.FalsePositives(name="fp"),
    keras.metrics.TruePositives(name="tp"),
    keras.metrics.FalseNegatives(name="fn"),
    'accuracy',
    keras.metrics.SpecificityAtSensitivity(0.5, name="ss")
]

# compile the keras mode
model.compile(
    optimizer=keras.optimizers.SGD(learning_rate=0.1), 
    loss='binary_crossentropy',
    metrics=metrics
)

callbacks = [keras.callbacks.ModelCheckpoint("bankruptcy_model_at_epoch_{epoch}.h5", monitor="val_loss", save_best_only=True, mode="min")]

# fit the keras model on the dataset
model.fit(X_train, y_train, epochs=20, batch_size=20, verbose=1,validation_data=(X_test, y_test), callbacks=callbacks)

# Generate generalization metrics
scores_test = model.evaluate(X_test, y_test, verbose=0)
scores_train = model.evaluate(X_train, y_train, verbose=0)
print(f'Score for fold: {model.metrics_names[0]} of {scores_test[0]}; {model.metrics_names[1]} of {scores_test[1]};{model.metrics_names[2]} of {scores_test[2]};{model.metrics_names[3]} of {scores_test[3]}; {model.metrics_names[4]} of {scores_test[4]}; {model.metrics_names[5]} of {scores_test[5]}; {model.metrics_names[6]} of {scores_test[6]};%')
print(f'Score for fold: {model.metrics_names[0]} of {scores_train[0]}; {model.metrics_names[1]} of {scores_train[1]}; {model.metrics_names[2]} of {scores_train[2]};{model.metrics_names[3]} of {scores_train[3]}; {model.metrics_names[4]} of {scores_train[4]}; {model.metrics_names[5]} of {scores_train[5]}; {model.metrics_names[6]} of {scores_train[6]};%')


inactive firms in the test: 2971
active firms in the test: 8122
Epoch 1/20
2219/2219 [==============================] - 9s 3ms/step - loss: 0.5085 - tn: 35280.0000 - fp: 9093.0000 - tp: 35278.0000 - fn: 9095.0000 - accuracy: 0.7953 - ss: 0.8736 - val_loss: 0.5925 - val_tn: 8122.0000 - val_fp: 2971.0000 - val_tp: 8122.0000 - val_fn: 2971.0000 - val_accuracy: 0.7322 - val_ss: 0.8091
Epoch 2/20
2219/2219 [==============================] - 7s 3ms/step - loss: 0.5048 - tn: 35286.0000 - fp: 9087.0000 - tp: 35287.0000 - fn: 9086.0000 - accuracy: 0.7952 - ss: 0.8788 - val_loss: 0.5955 - val_tn: 8122.0000 - val_fp: 2971.0000 - val_tp: 8122.0000 - val_fn: 2971.0000 - val_accuracy: 0.7322 - val_ss: 0.8133
Epoch 3/20
2219/2219 [==============================] - 7s 3ms/step - loss: 0.5046 - tn: 35285.0000 - fp: 9088.0000 - tp: 35287.0000 - fn: 9086.0000 - accuracy: 0.7952 - ss: 0.8802 - val_loss: 0.5888 - val_tn: 8122.0000 - val_fp: 2971.0000 - val_tp: 8122.0000 - val_fn: 2971.0000 - val_accuracy: 

In [14]:
# Fold 1
weights = []
for layer in model.layers:
    weights.append(layer.get_weights())
    
weights

[[array([[-0.13323274, -0.01587858,  0.25812644],
         [ 0.34199062,  0.21233985,  0.09598412],
         [ 0.61331713,  0.11593663, -0.00855894],
         [-0.16757591, -0.03455882, -0.03343703],
         [-0.20780002, -0.02287534, -0.01746157]], dtype=float32),
  array([ 0.01425134, -0.04690924,  0.52659494], dtype=float32)],
 [array([[0.01188125, 1.3818846 ],
         [0.6813046 , 0.50623465],
         [0.5029739 , 0.05597441]], dtype=float32),
  array([-1.4655936,  1.0470715], dtype=float32)]]

In [16]:
# Fold 2
weights = []
for layer in model.layers:
    weights.append(layer.get_weights())
    
weights

[[array([[-0.13437355,  0.18848298,  0.09635779],
         [-0.02151761,  0.13730332, -0.00332905],
         [ 0.00076678,  0.43118197, -0.07082431],
         [-0.48318726, -0.02041665, -0.04957085],
         [-0.622207  , -0.08236584,  0.27510428]], dtype=float32),
  array([ 0.29299217,  0.27390558, -0.01094244], dtype=float32)],
 [array([[0.02729873, 0.6263127 ],
         [0.5628098 , 0.04298412],
         [0.00139782, 0.25115007]], dtype=float32),
  array([-1.3538276,  0.9417543], dtype=float32)]]

In [17]:
# Fold 3
weights = []
for layer in model.layers:
    weights.append(layer.get_weights())
    
weights

[[array([[-0.21669145, -0.01740701,  0.26906788],
         [ 0.04738224, -0.04576507,  0.08138926],
         [ 0.30304495, -0.07604489,  0.38432434],
         [-0.41211295, -0.1569459 , -0.07364773],
         [-0.3265473 ,  0.08062775, -0.03329084]], dtype=float32),
  array([0.29072478, 0.26665854, 0.33823636], dtype=float32)],
 [array([[0.01944366, 0.6831187 ],
         [0.00782989, 0.28785414],
         [0.5510768 , 0.06649121]], dtype=float32),
  array([-1.5284808,  0.9988784], dtype=float32)]]

In [11]:
# Fold 4
weights = []
for layer in model.layers:
    weights.append(layer.get_weights())
    
weights


[[array([[ 0.33727452,  0.03316251, -0.31057498],
         [ 0.29264373,  0.01502134, -0.03074162],
         [ 0.3010764 ,  0.08709689, -0.030279  ],
         [ 0.08772742, -0.00993394, -0.47370517],
         [-0.08971967, -0.13417539, -0.33026785]], dtype=float32),
  array([0.4206048 , 0.0091901 , 0.33912128], dtype=float32)],
 [array([[5.4363090e-01, 8.8856518e-03],
         [1.2743917e-01, 3.0291453e-04],
         [6.9049299e-03, 5.8369732e-01]], dtype=float32),
  array([-1.5191718,  1.0645326], dtype=float32)]]

In [11]:
# Fold 5
weights = []
for layer in model.layers:
    weights.append(layer.get_weights())
    
weights

[[array([[ 0.22876817, -0.37973845,  0.05709085],
         [ 0.05080106,  0.3315255 ,  0.04359794],
         [ 0.16365135, -0.20637338,  0.08578772],
         [-0.20654131, -0.6608633 ,  0.11495377],
         [-0.00343514, -0.43067753, -0.01039881]], dtype=float32),
  array([0.30684084, 0.4105214 , 0.07280952], dtype=float32)],
 [array([[7.9296511e-01, 4.8704186e-01],
         [5.6018931e-04, 8.7540507e-01],
         [1.4588310e-01, 1.0256395e-02]], dtype=float32),
  array([-1.6705008,  0.8597965], dtype=float32)]]